Exercise 4 - Rasterizing Input Data and Packaging Imagery
---

In this last exercise we will rasterize the input data, create a JSON file which defines the mapping of raster values to class, and crop our imagery to the extent of the final label raster. This first cell is responsible for rasterizing the input data and creating the raster value mapping file.

In [1]:
import rasterio
from rasterio.profiles import DefaultGTiffProfile
from rasterio.crs import CRS as rasterioCRS
from rasterio import features
import rasterio.mask
import fiona
import glob
import geopandas as gpd
from shapely.geometry import GeometryCollection, box
import math
import os
import json

RESOLUTION = 10 # Spatial Resolution in Meters of the Imagery
LABEL_COLUMN = 'CROPS' # Column for which to Rasterize

DATASET_DIRECTORY = '../data/dataset'

if not os.path.exists(DATASET_DIRECTORY):
    os.makedirs(DATASET_DIRECTORY)

labels = gpd.read_file('../data/labels.geojson')

bbox = GeometryCollection([ geom for geom in labels.geometry ]).bounds
bounds = box(bbox[0], bbox[1], bbox[2], bbox[3])

width = math.ceil(abs(bbox[0] - bbox[2]) / RESOLUTION)
height = math.ceil(abs(bbox[1] - bbox[3]) / RESOLUTION)

all_labels = list(labels[LABEL_COLUMN].unique())

def raster_label(row):
    return all_labels.index(row[LABEL_COLUMN]) + 1

labels['RASTER_LABEL'] = labels.apply(lambda row: raster_label(row), axis=1)

label_shapes = [ (geom, v) for geom, v in zip(labels.geometry, labels.RASTER_LABEL) ]

label_key = {
    '0': 'No Data'
}

for i, l in enumerate(all_labels):
    label_key[str(i+1)] = l

with open(f'{DATASET_DIRECTORY}/labels.json', 'w') as f:
    f.write(json.dumps(label_key))

profile = DefaultGTiffProfile(
    count=1,
    width=width,
    height=height
)
profile['crs'] = rasterioCRS.from_epsg(str(labels.crs).replace('epsg:', ''))
profile['transform'] = rasterio.transform.from_origin(
    bbox[0],
    bbox[3],
    RESOLUTION,
    RESOLUTION
)

profile['nodata'] = 0

with rasterio.open(f'{DATASET_DIRECTORY}/labels.tif', 'w', **profile) as dst_dataset:
    image = features.rasterize(
        label_shapes,
        all_touched=True,
        fill=0,
        out_shape=dst_dataset.shape,
        transform=dst_dataset.transform
    )
    dst_dataset.write(image, indexes=1)

Crop Imagery to Input Data
---
Next, we will need to iterate through all of our imagery files, crop them to the extent of the label raster, and save them in a directory with the label raster.

In [2]:
files = glob.glob('../data/imagery/**/*.tif')
for fname in files:
    scene_id = fname.split('/')[-2]
    output_fname = fname.split('/')[-1]
    
    output_path = f'{DATASET_DIRECTORY}/imagery'
    if not os.path.exists(output_path):
        os.makedirs(output_path)
        
    output_fname = f'{output_path}/{scene_id}_{output_fname}'
    
    with rasterio.open(fname) as src:
        out_image, out_transform = rasterio.mask.mask(src, [bounds], crop=True)
        out_meta = src.meta
        
    out_meta.update({"driver": "GTiff",
                 "height": height,
                 "width": width,
                 "transform": out_transform})

    with rasterio.open(output_fname, "w", **out_meta) as dest:
        dest.write(out_image)

Done!
---

You now have a complete training dataset from just a simple shapefile of data collected in the field. The label raster, label raster value mapping, and cropped imagery are all located within the `data/dataset/` directory. In a follow up lecture you will learn how to create a [STAC](http://stacspec.org/) catalog of this dataset which allows for easy importing into [Radiant MLHub](https://mlhub.earth/).